# Hermite shape functions in 1D

Let $T = [0,1]$. We choose the monomial basis $G=\{1, x, x^2, x^3\}$ of $P_3(T)$ to express the shape functions in. The four degrees of freedom $L = \{l_i\}_{i=1}^4 \subset P_3(T)'$ are point evaluation and differentiation at $0$ and $1$. We build the Vandermonde matrix $V_{i j} = L_i(G_j)$ and invert it. The columns of $V^{-1}$ are the coefficients of the shape functions expressed in the basis $G$. The manual computation is easy in 1D, but since we will do it in 2D and 3D as well, we use Automatic Differentiation and a bit of Python to do the job.

In [ ]:
from __future__ import print_function
import autograd as ad
import autograd.numpy as np

# Partial derivatives
def dx(f):
    return ad.grad(f, 0)
def dy(f):
    return ad.grad(f, 1)
def dz(f):
    return ad.grad(f, 2)

In [ ]:
def create_shape_functions_code(basis, basis_strings, dofs, varname='sh', num_constraints=0):
    """Create python code with shape functions.
    This can be used for plotting or be pasted into ffc/tests/unit/test_elements.py.

    Arguments:
    ----------
        basis: list of callables. This is the polynomial basis in which 
               to express the shape functions.
                   E.g. [lambda x: 1, lambda x: x]
        basis_strings: string representation of the callables used to
                       output the code.
        dofs: list of callables. This are the linear functionals whose dual
              basis will be the shape functions. They should be unisolvent.
        varname: name for the variable defined in the code returned.
        num_constraints: Number of constraints in the dofs

    Returns:
    -------
        A string of python code which can be exec()'d or pasted elsewhere
        containing an assigment:
        
            varname = [lambda x: ..., lambda x: ..., ...]
            
        The list will be of length = len(dofs), but the last num_constraints
        will be 0.
    """
    
    n = len(basis)
    assert n == len(dofs) == len(basis_strings), "Dimensions don't match."

    # Build Vandermonde matrix
    V = np.zeros((n, n))
    for i, j in np.ndindex(V.shape):
        V[i,j] = dofs[i](basis[j])

    # Invert and read linear combinations of basis functions in G
    # from the columns
    Vinv = np.linalg.inv(V)
    M = np.zeros_like(Vinv)
    n = len(dofs)-num_constraints
    M[:,:n] = Vinv[:, :n]

    s = "%s = [" % varname
    newline = ",\n      "
    for j in range(n):
        lam = "lambda x: "
        l = []
        for i, basis_elem in enumerate(basis_strings):
            coeff = M[i,j]
            if not np.isclose(coeff, 0):
                sign = "+" if coeff > 0 else "-"
                coeff = np.abs(coeff)
                prefix = "%s %f *" % (sign, coeff) if not np.isclose(coeff, 1)\
                                                   else sign
                l.append(prefix)
                l.append(basis_elem)
        if not l:
            lam = ""
        elif l[0] == '+': # Remove unnecessary sign (confuses autograd)
            l.pop(0) 
        s += lam + " ".join(l) + newline
    s = s[:-len(newline)]    # Remove last ",\n      "
    return s + "]"

def hermite_shapes_1d(varname='sh1'):
    """ Returns the code for the Hermite shape functions in 1D

    Returns:
    --------
        String with List of callables (of one variable) with the shape functions.
    """
    # Monomial basis for $P_3(R)$
    G = [lambda x: 1, lambda x: x, lambda x: x**2, lambda x: x**3]

    # String representation of monomial basis
    S = ["1", "x", "x**2", "x**3"]

    # Hermite degrees of freedom on the reference interval [(0,0),(1,0)]:
    # Point evaluation and partial derivatives at each endpoint
    L = [lambda f: f(0.), lambda f: dx(f)(0.), lambda f: f(1.), lambda f: dx(f)(1.)]
    
    return create_shape_functions_code(G, S, L, varname)

In [ ]:
code = hermite_shapes_1d('sh')
print(code)
exec(code)

In [ ]:
import matplotlib.pyplot as pl
%matplotlib inline

pl.figure(figsize=(10,4))
pl.subplot(1,2,1)
xx = np.linspace(0, 1, 100)
for i, f in enumerate(sh):
    pl.plot(xx, [f(x) for x in xx], label='sh%d' % i)
_ = pl.legend()

pl.subplot(1,2,2)
for i, f in enumerate(sh):
    pl.plot(xx, [dx(f)(x) for x in xx], label='sh%d\'' % i)
_ = pl.legend()

# Hermite shape functions in 2D

Now $G=\{1, x, y, x^2, x y, y^2, x^3, x^2 y, x y^2, y^3\}$.

In [ ]:
def hermite_shapes_2d(varname='sh2'):
    """ Returns python code for the Hermite shape functions in 2D.

    Returns:
    --------
        List of callables (of two variables) with the shape functions.
    """
    # Monomial basis for $P_3(R^2)$
    G = [lambda x,y: 1,
         lambda x,y: x, lambda x,y: y,
         lambda x,y: x**2, lambda x,y: x*y, lambda x,y: y**2,
         lambda x,y: x**3, lambda x,y: x**2*y, lambda x,y: x*y**2, lambda x,y: y**3]

    # String representation of monomial basis for the code output
    S = ["1", "x[0]", "x[1]", "x[0]**2", "x[0]*x[1]", "x[1]**2",
         "x[0]**3", "x[0]**2*x[1]", "x[0]*x[1]**2", "x[1]**3"]

    # Hermite degrees of freedom on the reference triangle [(0,0),(1,0),(0,1)]:
    # point evaluation and partial derivatives at each vertex, plus evaluation at the barycenter.
    L = [lambda f: f(0., 0.), lambda f: dx(f)(0., 0.), lambda f: dy(f)(0., 0.),
         lambda f: f(1., 0.), lambda f: dx(f)(1., 0.), lambda f: dy(f)(1., 0.),
         lambda f: f(0., 1.), lambda f: dx(f)(0., 1.), lambda f: dy(f)(0., 1.),
         lambda f: f(1./3., 1./3.)]

    return create_shape_functions_code(G, S, L, varname)

In [ ]:
code = hermite_shapes_2d('sh2')
print(code)
exec(code)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import matplotlib.pyplot as pl

def plot_2dshapes(shapes, num_points=100, savefigs=None,
                  colors=['red', 'blue', 'green']):
    """ Plots scalar (shape) functions over the reference unit triangle.

    Arguments:
    ----------
        shapes: one or more (zipped) arrays of functions to evaluate.
                Each function must take one numpy.ndarray of shape (2,)
                and return one scalar.
        num_points: number of evaluation points per dimension 
                    (we use a square grid restricted to the unit triangle).
        savefigs: It not None, a string like "hermite2d-w-%d.eps"
                  with exactly one format string "%d" in it.
        colors: List of matplotlib colors, one for each family of shapes.
    """
    X = np.linspace(0, 1, num_points)
    Y = np.linspace(0, 1, num_points)
    xx, yy = np.meshgrid(X, Y)
    # Test for zipped shape functions
    funs = np.array(shapes)
    if len(funs.shape) == 1:
        funs = funs[:, None]
    num_funs = funs.shape[1]
    zz = np.zeros((num_funs, xx.shape[0], xx.shape[1]))
    for i, ff in enumerate(funs):
        fig = pl.figure()
        ax = fig.gca(projection='3d', title="$\phi_{%d}$" % i)
        for j,m,n in np.ndindex(num_funs, num_points, num_points):
            zz[j,m,n] = ff[j]([xx[m,n], yy[m,n]]) if n <= num_points - m\
                                                  else np.nan
        vmin, vmax = np.min(zz[~np.isnan(zz)]), np.max(zz[~np.isnan(zz)])
        for k in range(num_funs):
            ax.plot_wireframe(xx, yy, zz[k], color=colors[k],
                              linewidth=0.2, rstride=2, cstride=2)
            #ax.plot_surface(xx, yy, zz, cmap=cm.bone, linewidth=0,
            #                rstride=5, cstride=5, vmin=-0.5, vmax=1.0)
        
        # Get rid of the panes
        white = (1.0, 1.0, 1.0, 0.0)
        ax.w_xaxis.set_pane_color(white)
        ax.w_yaxis.set_pane_color(white)
        ax.w_zaxis.set_pane_color(white)

        ax.xaxis.set_ticks([0., 1.])
        ax.yaxis.set_ticks([0., 1.])
        ax.zaxis.set_ticks([vmin, 0, vmax])
        if savefigs is not None:
            pl.savefig(savefigs % i)

In [ ]:
plot_2dshapes(sh2)

# Hermite shape functions in 3D

Now $G=\{1, x, y, z, x^2, y^2, z^2, x y, x z, y z, x^2 y, x^2 z, y^2 z, x y^2, x z^2, y z^2, x y z, x^3, y^3, z^3\}$, i.e. $\text{dim}\ P_3(\mathbb{R}^3) = 20$.

In [ ]:
def hermite_shapes_3d(varname='sh3'):
    """ Returns python code for the Hermite shape functions in 3D.

    Returns:
    --------
        List of callables (of three variables) with the shape functions.
    """    
    # Monomial basis for $P_3(R^3)$
    G = [lambda x,y,z: 1,
         lambda x,y,z: x,      lambda x,y,z: y,      lambda x,y,z: z,
         lambda x,y,z: x**2,   lambda x,y,z: y**2,   lambda x,y,z: z**2,
         lambda x,y,z: x*y,    lambda x,y,z: y*z,    lambda x,y,z: x*z,
         lambda x,y,z: x**2*y, lambda x,y,z: x**2*z, lambda x,y,z: y**2*z,
         lambda x,y,z: x*y**2, lambda x,y,z: x*z**2, lambda x,y,z: y*z**2,
         lambda x,y,z: x*y*z,
         lambda x,y,z: x**3,   lambda x,y,z: y**3,   lambda x,y,z: z**3]

    # String representation of monomial basis for the code output
    S = ["1",
         "x[0]", "x[1]", "x[2]" ,
         "x[0]**2", "x[1]**2", "x[2]**2",
         "x[0]*x[1]", "x[1]*x[2]", "x[0]*x[2]",
         "x[0]**2*x[1]", "x[0]**2*x[2]", "x[1]**2*x[2]",
         "x[0]*x[1]**2", "x[0]*x[2]**2", "x[1]*x[2]**2",
         "x[0]*x[1]*x[2]",
         "x[0]**3", "x[1]**3", "x[2]**3"]

    # Hermite degrees of freedom on the reference tetrahedron [(0, 0, 0), (1, 0, 0), (0, 1, 0), (0, 0, 1)]:
    # point evaluation and partial derivatives at each vertex, plus evaluation at the barycenter of each face.
    # NOTE that we need to use the same ordering as in the FIAT code in order for the tests to pass.
    # To see in which sequence the barycenters were created use:
    #import FIAT
    #e = FIAT.CubicHermite(FIAT.reference_element.UFCTetrahedron(), 3)
    #ll = e.dual_basis()
    #for l in ll:
    #    print(l.pt_dict.keys())

    L = [lambda f: f(0., 0., 0.), lambda f: dx(f)(0., 0., 0.), lambda f: dy(f)(0., 0., 0.), lambda f: dz(f)(0., 0., 0.),
         lambda f: f(1., 0., 0.), lambda f: dx(f)(1., 0., 0.), lambda f: dy(f)(1., 0., 0.), lambda f: dz(f)(1., 0., 0.),
         lambda f: f(0., 1., 0.), lambda f: dx(f)(0., 1., 0.), lambda f: dy(f)(0., 1., 0.), lambda f: dz(f)(0., 1., 0.),
         lambda f: f(0., 0., 1.), lambda f: dx(f)(0., 0., 1.), lambda f: dy(f)(0., 0., 1.), lambda f: dz(f)(0., 0., 1.),
         lambda f: f(1./3., 1./3., 1./3.),
         lambda f: f(0., 1./3., 1./3.), lambda f: f(1./3., 0., 1./3.), lambda f: f(1./3., 1./3., 0.)]

    return create_shape_functions_code(G, S, L, varname)

In [ ]:
code = hermite_shapes_3d('sh')
print(code)
exec(code)

## Discrete Kirchhoff Triangles

In [ ]:
def kirchhoff_shapes_code(varname='kh'):
    """ Returns python code for Kirchhoff shape functions (Discrete Kirchhoff Elements).
    
    FIXME: does it?
    
    Returns:
    --------
        List of callables (of two variables) with the shape functions.
    """
    # Monomial basis for $P_3(R^2)$
    G = [lambda x,y: 1, lambda x,y: x, lambda x,y: y, lambda x,y: x**2,
         lambda x,y: x*y, lambda x,y: y**2, lambda x,y: x**3,
         lambda x,y: x**2*y, lambda x,y: x*y**2, lambda x,y: y**3]

    # String representation of monomial basis for the code output
    S = ["1", "x[0]", "x[1]", "x[0]**2", "x[0]*x[1]", "x[1]**2",
         "x[0]**3", "x[0]**2*x[1]", "x[0]*x[1]**2", "x[1]**3"]

    def constraint(f):
        xt, yt = 1./3, 1./3
        r = -f(xt,yt)
        for x,y in [(0.,0.), (1.,0.), (0.,1.)]:
            r += f(x,y) + dx(f)(x,y)*(xt-x) + dy(f)(x,y)*(yt-y)
        return r

    # 9 of the 10 Hermite degrees of freedom on the reference triangle:
    # point evaluation and partial derivatives at each vertex
    L = [lambda f: f(0., 0.), lambda f: dx(f)(0., 0.), lambda f: dy(f)(0., 0.),
         lambda f: f(1., 0.), lambda f: dx(f)(1., 0.), lambda f: dy(f)(1., 0.),
         lambda f: f(0., 1.), lambda f: dx(f)(0., 1.), lambda f: dy(f)(0., 1.),
         constraint]

    return create_shape_functions_code(G, S, L, varname, num_constraints=1)

In [ ]:
s = kirchhoff_shapes_code('kh')
print(s)
exec(s)

In [ ]:
plot_2dshapes(zip(sh2, kh))

# Interface

In [ ]:
def hermite_shapes(dim):
    if dim == 1:
        code = hermite_shapes_1d(varname='sh')
    elif dim == 2:
        code = hermite_shapes_2d(varname='sh')
    elif dim == 3:
        code = hermite_shapes_3d(varname='sh')
    else:
        raise ValueError("dim should be 1,2 or 3")
    exec(code)
    return sh

def kirchhoff_shapes():
    code = kirchhoff_shapes_code('kh')
    exec(code)
    return kh

In [ ]:
def __nbinit__():
    global __all__
    
    __all__ = ['hermite_shapes', 'kirchhoff_shapes', 'plot_2dshapes']

# Testing

In [ ]:
def pt(x,y):
    return np.array((x,y))  # setting dtype here annoys autograd

def compute_trafo(v1, v2, v3):
    """ Returns the affine trafo from the reference triangle
    to the triangle with vertices at v1, v2, v3.

    Arguments
    ---------
        v1, v2, v3: np.ndarrays of shape (2,)
    
    Returns
    -------
        A function accepting one np.ndarray of shape (2,)
        and returning another np.ndarray of shape (2,).
    """

    assert v1.shape == v2.shape == v3.shape == (2,),\
           "Wrong shapes"
    M = np.vstack((v2-v1, v3-v1)).T
    def trafo(x):
        return v1 + np.dot(M, x)
    return trafo

def compute_inverse_trafo(v1, v2, v3):
    """ Returns the affine trafo from the triangle with
    vertices at v1, v2, v3. to the reference triangle.
    
    Arguments
    ---------
        v1, v2, v3: np.ndarrays of shape (2,)
    
    Returns
    -------
        A function accepting one np.ndarray of shape (2,)
        and returning another np.ndarray of shape (2,).
    """

    assert v1.shape == v2.shape == v3.shape == (2,),\
           "Wrong shapes"
    Minv = np.linalg.inv(np.vstack((v2-v1, v3-v1)).T)
    def trafo(x):
        return np.dot(Minv, x-v1)
    return trafo

# Test trafos:
v1,v2,v3 = pt(0., 0.), pt(1., 0.), pt(1., 1)
F = compute_trafo(v1,v2,v3)
Finv = compute_inverse_trafo(v1,v2,v3)
np.alltrue([np.allclose(x, F(Finv(x))) and np.allclose(x, Finv(F(x)))
            for x in np.random.rand(20,2)])

In [ ]:
def hermite_shapes_2d_physical(v1, v2, v3):
    """ Return Hermite shape functions for a physical element.

    Arguments
    ---------
        v1,v2,v3: vertices of the triangle where the shape functions
                  will be supported.
    Returns
    -------
        List of callables each accepting a 2d point. They correspond, in order,
        to evaluation, dx, dy at v1, then v2 then v3, followed by a last evaluation
        at the barycenter.
    """
    
    F = compute_trafo(v1,v2,v3)
    Finv = compute_inverse_trafo(v1, v2, v3)
    phi_hat = hermite_shapes(dim=2)

    # Build Hermite transformation matrix
    J = ad.jacobian(F)(v1)  # TODO: assert this is constant
    H = np.identity(3)
    H[1:,1:] = J

    # Transform basis functions:
    phi = []
    #for i in range(9):
    #    phi.append(lambda x: np.dot(H, [f(F(x)) for f in phi_hat[i/3:3+i/3]])[i%3])
    phi.append(lambda x: np.dot(H, [f(Finv(x)) for f in phi_hat[0:3]])[0])
    phi.append(lambda x: np.dot(H, [f(Finv(x)) for f in phi_hat[0:3]])[1])
    phi.append(lambda x: np.dot(H, [f(Finv(x)) for f in phi_hat[0:3]])[2])
    phi.append(lambda x: np.dot(H, [f(Finv(x)) for f in phi_hat[3:6]])[0])
    phi.append(lambda x: np.dot(H, [f(Finv(x)) for f in phi_hat[3:6]])[1])
    phi.append(lambda x: np.dot(H, [f(Finv(x)) for f in phi_hat[3:6]])[2])
    phi.append(lambda x: np.dot(H, [f(Finv(x)) for f in phi_hat[6:9]])[0])
    phi.append(lambda x: np.dot(H, [f(Finv(x)) for f in phi_hat[6:9]])[1])
    phi.append(lambda x: np.dot(H, [f(Finv(x)) for f in phi_hat[6:9]])[2])
    # Check the values obtained in test_duality when Finv(x) is left out in the following:
    phi.append(lambda x: phi_hat[9](Finv(x)))
    
    return phi

def test_duality(funs, v1, v2, v3):
    v4 = 1/3.*(v1 + v2 + v3)
    
    L = [lambda f: f(v1), lambda f: dx(f)(v1)[0], lambda f: dx(f)(v1)[1],
         lambda f: f(v2), lambda f: dx(f)(v2)[0], lambda f: dx(f)(v2)[1],
         lambda f: f(v3), lambda f: dx(f)(v3)[0], lambda f: dx(f)(v3)[1],
         lambda f: f(v4)]

    n = len(funs)
    A = np.zeros((n, n))
    for i,j in np.ndindex(A.shape):
        A[i,j] = L[i](funs[j])

    if np.allclose(A, np.identity(n)):
        print("Everything ok.")
        return True
    else:
        print("Failed! Returning A")
        return np.round(A,1)
    #for x in ref_triangle:
    #    print("x = %s" % x)
    #    for i, f in enumerate(phi_hat):
    #        print("\tf_%d(x) = %.1f" % (i, f(x)))
    #        print("\tdf_%d_dx(x) = %.1f" % (i, dx(f)(x)[0]))
    #        print("\tdf_%d_dy(x) = %.1f" % (i, dx(f)(x)[1]))

test_duality(hermite_shapes(dim=2), pt(0., 0.), pt(1., 0.), pt(0., 1))
test_duality(hermite_shapes_2d_physical(v1,v2,v3), v1,v2,v3)
test_duality(kirchhoff_shapes(), pt(0., 0.), pt(1., 0.), pt(0., 1)) 

## Integration tests

Manual assembly of a piece of the mass matrix. Pure python and with so many indirections this is pathetically slooooow!

I don't even remember why this was necessary or what it shows...

In [ ]:
phi = hermite_shapes_2d_physical(v1,v2,v3)

from scipy.integrate import dblquad

def integ(funs, i, j):
    # dblquad requires y first, x second
    def fun(y,x):
        return np.dot(dx(funs[i])(pt(x,y)), dx(funs[j])(pt(x,y)))
    return fun

M = np.zeros((len(phi), len(phi)))
for i,j in np.ndindex(M.shape):
    quad = dblquad(integ(phi, i, j), 0, 1, lambda x:0., lambda x:x)
    assert quad[1] < 1e-10, "Integration error too high"
    M[i,j] = quad[0]

In [ ]:
np.diag(M).round(4)

In [ ]:
import dolfin as df  # don't import *: we have defined a dx() above

In [ ]:
V = df.FunctionSpace(df.UnitSquareMesh(1,1), 'Hermite', 3)
u = df.TrialFunction(V)
v = df.TestFunction(V)
#A = df.assemble(df.inner(df.div(df.grad(u)), df.div(df.grad(v)))*df.dx)
A = df.assemble(df.inner(df.grad(u), df.grad(v))*df.dx)

In [ ]:
dm = V.dofmap()
loc2glob = dm.cell_dofs(0) # global indices of shape funcs local to cell 0
np.diag(A.array())[loc2glob].round(4)

WHAT THE FUCCCCKKCKKKKKCKCKKKCKCCC**ASDå@∆∆¡¿≠“¶∆åÍ™¬°*!*!**!*!ÖDASDFAdF gnggñgnngng~©©~~gnkklll

In [ ]:
# Just to be sure...
Cell(V.mesh(), 0).get_vertex_coordinates().reshape((-1,2))